In [5]:
"""
-- Created by: Ashok Kumar Pant
-- Email: asokpant@gmail.com
-- Created on: 18/05/2025
"""
import os

from langchain.chains import RetrievalQA
from langchain_anthropic import ChatAnthropic
from langchain_community.llms import LlamaCpp, HuggingFaceHub
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import PromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_openai import ChatOpenAI


# Dependencies:
# pip install langchain faiss-cpu llama-cpp-python
# pip install -U langchain-community langchain_google_genai
# pip install -U langchain_huggingface


class LLMFactory:
    @staticmethod
    def load_llm(llm_type: str = "llamacpp", model_name: str = None, api_key: str = None, ):
        if model_name is None:
            raise ValueError("Model name must be provided.")
        if llm_type == "openai":
            return ChatOpenAI(model=model_name, api_key=api_key)
        elif llm_type == "anthropic":
            return ChatAnthropic(model=model_name, api_key=api_key)
        elif llm_type == "gemini":
            return ChatGoogleGenerativeAI(model=model_name, google_api_key=api_key)
        elif llm_type == "huggingfacehub":
            return HuggingFaceHub(repo_id=model_name, huggingfacehub_api_token=api_key)
        elif llm_type == "llamacpp":
            return LlamaCpp(
                model_path=model_name,
                n_gpu_layers=-1,
                max_tokens=500,
                n_ctx=2048,
                seed=42,
                verbose=False
            )
        else:
            raise ValueError(f"Unsupported LLM type: {llm_type}")


class RAGIndexer:
    def __init__(self, embedding_model_name: str = "thenlper/gte-small", index_path: str = "kullm.faiss.db"):
        self.embedding_model_name = embedding_model_name
        self.index_path = index_path
        self.embedding_model = self._load_embedding_model()
        self.vector_db = self._load_or_create_index()

    def _load_embedding_model(self):
        print("[Indexer] Loading embedding model...")
        return HuggingFaceEmbeddings(model_name=self.embedding_model_name)

    def _load_or_create_index(self):
        if os.path.exists(self.index_path):
            print(f"[Indexer] Loading FAISS index from '{self.index_path}'")
            return FAISS.load_local(self.index_path, self.embedding_model)
        else:
            print("[Indexer] Creating a new FAISS index...")
            return FAISS.from_texts([""], self.embedding_model)


    def add_documents(self, texts: list):
        print(f"[Indexer] Adding {len(texts)} document(s) to the index...")
        self.vector_db.add_texts(texts)
        self.vector_db.save_local(self.index_path)

    def get_retriever(self):
        return self.vector_db.as_retriever()


class RAGModel:
    def __init__(self, llm_type: str, llm_model_name: str, embedding_model_name: str,
                 index_path: str = "kullm.faiss.db", api_key: str = None):
        self.llm_model_name = llm_model_name
        self.embedding_model_name = embedding_model_name
        self.index_path = index_path

        self.llm = LLMFactory.load_llm(llm_type, llm_model_name, api_key)
        self.prompt_template = self._default_prompt_template()
        self.indexer = RAGIndexer(self.embedding_model_name, self.index_path)
        self.rag_chain = self._build_rag_chain()

    def _load_llm(self):
        print("[INFO] Loading generation model...")
        return LlamaCpp(
            model_path=self.llm_model_name,
            n_gpu_layers=-1,
            max_tokens=500,
            n_ctx=2048,
            seed=42,
            verbose=False
        )

    def _default_prompt_template(self):
        template = """<|user|>
Relevant information:
{context}
Provide a concise answer to the following question using the relevant information provided above:
{question}<|end|>
<|assistant|>"""
        return PromptTemplate(template=template, input_variables=["context", "question"])

    def _build_rag_chain(self):
        print("[INFO] Building RAG chain...")
        return RetrievalQA.from_chain_type(
            llm=self.llm,
            chain_type='stuff',
            retriever=self.indexer.get_retriever(),
            chain_type_kwargs={"prompt": self.prompt_template},
            verbose=False
        )

    def add_documents(self, texts: list):
        self.indexer.add_documents(texts)
        self.rag_chain.retriever = self.indexer.get_retriever()

    def ask(self, query: str) -> str:
        print(f"[RAGModel] Query: {query}")
        response = self.rag_chain.invoke(query)
        return response.get("result", "[No answer generated]")

In [6]:
# Load RAG model
llm_model_name = "/Users/ashokpant/.cache/lm-studio/models/nomic-ai/nomic-embed-text-v1.5-GGUF/nomic-embed-text-v1.5.Q8_0.gguf"
embedding_model_name = "thenlper/gte-small"

rag_system = RAGModel(llm_type="llamacpp", llm_model_name=llm_model_name, embedding_model_name=embedding_model_name)

llama_context: n_batch is less than GGML_KQ_MASK_PAD - increasing to 64
ggml_metal_init: skipping kernel_get_rows_bf16                     (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_f32                   (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_f32_1row              (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_f32_l4                (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_bf16                  (not supported)
ggml_metal_init: skipping kernel_mul_mv_id_bf16_f32                (not supported)
ggml_metal_init: skipping kernel_mul_mm_bf16_f32                   (not supported)
ggml_metal_init: skipping kernel_mul_mm_id_bf16_f32                (not supported)
ggml_metal_init: skipping kernel_flash_attn_ext_bf16_h64           (not supported)
ggml_metal_init: skipping kernel_flash_attn_ext_bf16_h80           (not supported)
ggml_metal_init: skipping kernel_flash_attn_ext_bf16_h96           (not supported)
ggml_metal_init

[Indexer] Loading embedding model...
[Indexer] Creating a new FAISS index...
[INFO] Building RAG chain...


In [ ]:
# Define your knowledge base
knowledge_base_texts = [
    "The Income generated by the film in 2014 was over $677 million worldwide.",
    "This made it the tenth-highest grossing film of that year.",
    "With subsequent releases, total earnings reached approximately $773 million.",
    "The release format transitioned from film stock to digital projectors in the US."
]

# Add documents and ask a question
rag_system.add_documents(knowledge_base_texts)

question = "Income generated"
answer = rag_system.ask(question)
print("\n[ANSWER]", answer)

# Add new documents to the index
new_documents = [
    "The film was released in 2014.",
    "It was a box office success."
]
rag_system.add_documents(new_documents)
print("\n[INFO] New documents added to the index.")

# Ask another question
question = "What year was the film released?"
answer = rag_system.ask(question)
print("\n[ANSWER]", answer)